In [1]:
# prereq.
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
# helper: normalize

def normalize_df(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return pd.DataFrame(x_scaled)

In [3]:
dataset = pd.read_csv('creditcard.csv')

In [4]:
train = dataset.sample(frac=0.98,random_state=200) # split in 98% and 2%, 98% for train and 2% for dev set
cross = dataset.drop(train.index)

In [5]:
Y = train.iloc[:, -1:]
Y_cross = cross.iloc[:, -1:]

In [6]:
dataset_X_normed = normalize_df(dataset.iloc[:, 1:-1])
dataset_Y = dataset.iloc[:, -1:]

X = dataset_X_normed.sample(frac=0.98,random_state=200) # split in 98% and 2%, 98% for train and 2% for dev set
X_cross = dataset_X_normed.drop(X.index)


In [7]:
data_type =  np.dtype(np.float64)

In [8]:
# convert to numpy

x = np.asarray(X.to_numpy(), dtype=data_type)
y = np.asarray(Y.to_numpy(), dtype=data_type)
x_c = np.asarray(X_cross.to_numpy(), dtype=data_type)
y_c = np.asarray(Y_cross.to_numpy(), dtype=data_type)

In [9]:
# helper: compute miu and sigma

def estimateGaussian(X):
    m, n = X.shape # get the length of train set and the nr of features
    
    miu = np.zeros((n, 1), dtype=data_type)
    sigma = np.zeros((n, n), dtype=data_type)
    miu.reshape(n,1)
    sigma.reshape(n,n)
    
    for i in range(0,n):
        for j in range(0, m):
            miu[i] += X[j][i]
        miu[i] = miu[i] / m    
            
    for i in range(0, m):
        aux = X[i].reshape(n,1)
        sigma += np.dot((aux - miu), (aux - miu).T)
    sigma = sigma / m
    
    return miu, sigma
        

In [10]:
miu, sigma = estimateGaussian(x) # estimate miu and sigma for training dataset

In [11]:
# export miu and sigma to df
miu_df = pd.DataFrame(miu, columns=['miu']) 
sigma_df = pd.DataFrame(sigma)

In [12]:
# export as csv miu and sigma bcs, is hard to compute all the time them
miu_df.to_csv('miu_train.csv')
sigma_df.to_csv('sigma_train.csv')

In [13]:
# multivariateGaussian
from numpy.linalg import inv, det, matrix_rank

def multivariateGaussian(X, miu, sigma):
    inv_sigma = inv(sigma)
    det_sigma = det(sigma)
    
    m, n = X.shape
    predict = np.zeros((m, 1), dtype=data_type)
    coef = 1 / (((2*np.pi)**(n / 2)) * np.sqrt(det_sigma))
    
    for i in range(0, m):
        aux = X[i].reshape(n,1)
        predict[i] =  coef * np.exp(-1/2 * np.dot(np.dot((aux-miu).T, inv_sigma), (aux-miu)))
    
    return predict

In [14]:
predicted = multivariateGaussian(x, miu, sigma)

In [15]:
def select_the_treshold(y, predicted, an, nonan, dec_step=0):
    best_epsilon = 0
    best_F1 = 0
    F1 = 0
    best_acc = 0
    
    epsilon = float(1.0)
    
    len_pred = len(predicted)
    iter = 0 # number of iterations
    penalization = len_pred / len(np.where(y==1)[0]) # penalization for F1 score
    
    out_30 = 0 # exit loop if done
    F1_decrease_5 = 0 # exit loop if done
    
    while True: 

        tp = fp = fn = 0
        acc = 0
        
        for i in range(len_pred):
            if (predicted[i][0] < epsilon) == True and y[i] == 1:
                tp += 1
                acc += 1
                
            if (predicted[i][0] < epsilon) == True and y[i] == 0:
                fp += penalization * np.abs(nonan + 1)
                
            if (predicted[i][0] < epsilon) == False and y[i] == 1:
                fn += penalization * np.abs(an + 1)
                
            if (predicted[i][0] < epsilon) == False and y[i] == 0:
                acc += 1
                
        acc = acc / len_pred
        prec = tp / (tp + fp)
        rec = tp / (tp + fn)
        F1_old = F1
        F1 = 2 * prec * rec / (prec + rec )
        
        if F1 > best_F1:
            best_F1 = F1
            best_epsilon = epsilon
            print(epsilon, F1, '------------', acc)
            best_acc = acc
            
        else:
            out_30 += 1
            if out_30 == 30: # exit, if after 30 iteration, there wasn't found a new max_F1
                print(epsilon, F1, '------------', acc, '20:iterrr', iter)
                break
        
        if F1_old > F1:
            F1_decrease_5 += 1
            if F1_decrease_5 == 5: # breake if F1 decrease 3 iteration consecutive
                break
        else:
            F1_decrease_5 = 0
            
        if dec_step:
            go_down = 10 / (1 + (iter+1)**(1/5)) # step
        else:
            go_down = 10
        epsilon = float(epsilon / go_down)
        
        iter += 1
        print(iter, epsilon, F1, prec, rec, acc)
    
    return best_epsilon, best_F1

In [16]:
epsilon, F1 = select_the_treshold(y, predicted, 0, 0, 1) # an penalize if wasn't identif corect anomaly and nonan penalize when target a non anomaly as anomaly

1.0 0.00028226318132066935 ------------ 0.9859625740296871
1 0.2 0.00028226318132066935 0.0001475528499993119 0.0032430349449467225 0.9859625740296871
0.2 0.0002879112692685118 ------------ 0.9862814435833772
2 0.0429739670999407 0.0002879112692685118 0.00015070578876336056 0.0032139929029092102 0.9862814435833772
0.0429739670999407 0.000291803594110492 ------------ 0.9865501538814306
3 0.009650796751435617 0.000291803594110492 0.0001529716304768912 0.0031569111256168365 0.9865501538814306
0.009650796751435617 0.0002960610884160935 ------------ 0.9867436252960292
4 0.002238509941021626 0.0002960610884160935 0.00015531333308106693 0.0031569111256168365 0.9867436252960292
0.002238509941021626 0.0002991878468866214 ------------ 0.98696575914242
5 0.0005327048504124958 0.0002991878468866214 0.0001571758598795365 0.0031011277590794564 0.98696575914242
0.0005327048504124958 0.0003051412685898041 ------------ 0.9872201382245773
6 0.00012949890207075774 0.0003051412685898041 0.0001604652747797

1.0940984502841596e-29 0.00042154872560836376 ------------ 0.993092353938039
54 3.523700553427639e-30 0.00042154872560836376 0.00024279906878701426 0.0015980105809703829 0.993092353938039
3.523700553427639e-30 0.00042231078021385084 ------------ 0.9931640100175199
55 1.1377350366030157e-30 0.00042231078021385084 0.00024392033016531721 0.0015719574758653074 0.9931640100175199
1.1377350366030157e-30 0.0004242618854040295 ------------ 0.9932249176850787
56 3.682682010116859e-31 0.0004242618854040295 0.00024553937316904625 0.0015590824822558544 0.9932249176850787
3.682682010116859e-31 0.00042534089056908464 ------------ 0.9932714941367413
57 1.1949515523699267e-31 0.00042534089056908464 0.0002465842723092929 0.0015463069665924225 0.9932714941367413
58 3.886708833583361e-32 0.0004243379707361351 0.0002465614816875515 0.0015210497927962007 0.9933144877844299
3.886708833583361e-32 0.0004266232990673373 ------------ 0.9933503158241703
59 1.2671924143028864e-32 0.0004266232990673373 0.000248105

In [17]:
def test_the_epsilon(y, predicted, epsilon):
    acc = 0
    accy_1 = 0
    y_1 = 0
    nu_anomalii = 0
    y_0 = 0
    
    for i in range(len(predicted)):
        if y[i] == 1:
            y_1 += 1
        else:
            y_0 += 1
        
        if (predicted[i][0] < epsilon) == True and y[i] == 1:
                acc += 1
                accy_1 += 1
                
        if (predicted[i][0] < epsilon) == False and y[i] == 0:
                acc += 1
                
        if (predicted[i][0] < epsilon) == True and y[i] == 0:
                nu_anomalii += 1
                
                
                
                
    acc = acc / len(predicted)
    
    return str(acc), str(accy_1 / y_1), str(y_1) , str(accy_1), str(nu_anomalii), str(nu_anomalii / y_0 * 100)
    #return 'ACC: ' + str(acc), 'how meny anomaly did we found from the total of: ' + str(accy_1 / y_1), 'nr of anomaly: ' + str(y_1) , 'total of anomaly corectly found:' + str(accy_1), 'non anomaly targeted as anomaly: ' + str(nu_anomalii), 'non anomaly targeted as anomaly for the total of non anomaly: ' + str(nu_anomalii / y_0 * 100)

In [18]:
#miuc, sigmac = estimateGaussian(x_c)
predicted_c = multivariateGaussian(x_c, miu, sigma)

In [19]:
print(test_the_epsilon(y, predicted, epsilon), 'Non anomaly in dataset: ' + str(len(np.where(y==0)[1])))
print(epsilon, 'test')
print('------')
print(test_the_epsilon(y_c, predicted_c, epsilon), 'Non anomaly in dataset: ' + str(len(np.where(y_c==0)[1])))
print(epsilon, 'dev')

('0.9943570837408773', '0.44353182751540043', '487', '216', '1304', '0.4680142414149535') Non anomaly in dataset: 278624
1.074848348000021e-41 test
------
('0.9952598314606742', '0.4', '5', '2', '24', '0.42171850289931473') Non anomaly in dataset: 5691
1.074848348000021e-41 dev
